## The Generator

From Lewis et. al. (2020):

**2.3 Generator: BART**\
The generator component $p_{\theta}(y_i | x, z, y_{1:i-1})$ could be modelled using any encoder-decoder. We use BART-large [32], a pre-trained seq2seq transformer [58] with 400M parameters. To combine the input $x$ with the retrieved content $z$ when generating from BART, we simply concatenate them. BART was pre-trained using a denoising objective and a variety of different noising functions. It has obtained state-of-the-art results on a diverse set of generation tasks and outperforms comparably-sized T5 models [32]. We refer to the BART generator parameters as the parametric memory henceforth.

For our project we have many more options, including actual LLMs. The smallest "reasoning" LLMs usually have 7B parameters which means we need to quantize them, to 4-bits if we want to run them on a normal GPU. Some high-end cards might be able to run 8-bit quantized models. Any models larger than 7B will have to be run on some cloud. But even the free-tier T4 GPU on colab will run inte memory issues unless handled properly.

There are also smaller models specialized (finte-tuned) for RAG. DRAGON are 7B parameter models and BLING are smaller with between 1-7B parameters \[https://huggingface.co/llmware\]. According to https://medium.com/@darrenoberst/rag-instruct-capabilities-they-grow-up-so-fast-2647550cdc0a (from llmware) a 2.7B parameter model should suffice. The 2.7B parameter model is too big for me to run locally.

One option then is to quantize the 2.7B model. But first I test with a 7B model here on colab. Nope. Ran out of RAM. Testing the 2.7B model. Nope ran out of RAM.

When loading the model we can specify



In [1]:
# the last 4 installations are required as of now, restart session after install complete
#!pip install llmware                 # not needed now (might be a useful interface)
#!pip install sentencepiece           # needed for some tokenizers
#!pip install peft                    # performance efficient fine tuning (1 model)
#!pip install torch==2.1.0            #
#!pip install xformers==0.0.22.post4 --index-url https://download.pytorch.org/whl/cu118  # provides building blocks to some models
#!pip install bitsandbytes accelerate einops # for 4- and 8 bit quantization
#!pip install datasets                # access to the the document repository on huggingface

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00


In [1]:
#from transformers import AutoTokenizer, AutoModel, BartForConditionalGeneration, T5Tokenizer, T5ForConditionalGeneration
import transformers
import torch
from datasets import load_dataset
#from llmware.prompts import Prompt # seems like a user-friendly interface
#from peft import PeftModel, PeftConfig # for EverythingLM-13b-V3-peft


import pickle

models = [('facebook/bart-large', 'auto'), #https://huggingface.co/docs/transformers/v4.36.1/en/model_doc/bart
          ('google/flan-t5-base', 'T5-base'), #https://huggingface.co/docs/transformers/v4.36.1/en/model_doc/flan-t5
          ('google/flan-t5-small', 'T5-small'),
          ('facebook/bart-large-mnli', 'bart-mnli'), #https://huggingface.co/facebook/bart-large-mnli
          ('llmware/bling-falcon-1b-0.1', 'causal'), # SUCKS # requires llmware library(?) https://huggingface.co/llmware/bling-falcon-1b-0.1
          ('Intel/neural-chat-7b-v3-1', 'causal_4bit'), #https://huggingface.co/Intel/neural-chat-7b-v3-1
          ('devvanshhh/flan-xl-gen5', 'seq2seq'), #https://huggingface.co/devvanshhh/flan-xl-gen5?library=true
          ('llmware/bling-sheared-llama-2.7b-0.1', 'causal'),
          ('filipealmeida/Mistral-7B-Instruct-v0.1-sharded', 'causal_4bit'), # https://medium.datadriveninvestor.com/mistral-7b-on-free-colab-06b42d7b90e3
          ('vilsonrodrigues/falcon-7b-instruct-sharded', 'causal'), # causal_4bit https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded
          ('llmware/bling-sheared-llama-1.3b-0.1', 'causal'),
          ('mistralai/Mistral-7B-Instruct-v0.2', 'causal_4bit') # v0.2 ... OOM error when loading
          ]


In [2]:
selected_model = models[-1]

In [3]:
bling_models = ['llmware/bling-falcon-1b-0.1', 'llmware/bling-sheared-llama-1.3b-0.1']
intel_model = ['Intel/neural-chat-7b-v3-1']
mistral_models = ['filipealmeida/Mistral-7B-Instruct-v0.1-sharded', 'mistralai/Mistral-7B-Instruct-v0.2']

In [4]:
def load_HF_model(model_name, type='auto'):
  """ Takes a HuggingFace model name (str) and returns the model and
  corresponding tokenizer. Set type to match model to load weights on."""
  if type == 'causal':
    model = transformers.AutoModelForCausalLM.from_pretrained(model_name).to('cuda')
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
  elif type == 'T5-base':
    model = transformers.T5ForConditionalGeneration.from_pretrained(model_name).to('cuda')
    tokenizer = transformers.T5Tokenizer.from_pretrained(model_name)
  elif type == 'auto':
    model = transformers.AutoModel.from_pretrained(model_name).to('cuda')
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
  elif type == 'seq2seq':
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name).to('cuda')
    model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_name)
  elif type == 'causal_peft':
    config = PeftConfig.from_pretrained("totally-not-an-llm/EverythingLM-13b-V3-peft")
    model = transformers.AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-13b-hf")
    model = PeftModel.from_pretrained(model, "totally-not-an-llm/EverythingLM-13b-V3-peft")
  elif type == 'causal_4bit':
    bnb_config = transformers.BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True,
                                                 bnb_4bit_quant_type='nf4',
                                                 bnb_4bit_compute_dtype=torch.bfloat16)
    tokenizer = transformers.AutoTokenizer.from_pretrained(selected_model[0])
    model = transformers.AutoModelForCausalLM.from_pretrained(selected_model[0], trust_remote_code=True,
                                                              quantization_config=bnb_config,
                                                              device_map='auto')
  else:
    raise NotImplementedError("Model type not implemented.")
  # add attribute for model name
  model.name = model_name
  return model, tokenizer

In [5]:
# load model and tokenizer
model, tokenizer = load_HF_model(selected_model[0], selected_model[1])

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
print(model.name)

mistralai/Mistral-7B-Instruct-v0.2


In [7]:
# load documents and embeddings
dataset_documents = load_dataset("anordkvist/gu-course-syllabus")
dataset_embeddings = load_dataset("anordkvist/gu-course-syllabus-embeddings")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2975 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2971 [00:00<?, ? examples/s]

In [8]:
df = dataset_documents['train'].to_pandas()

In [13]:
# some parameters
MAX_NEW_TOKENS = 400
TEMP = 0.3
DO_SAMPLE = True
NUM_BEAMS = 2
SKIP_SPECIAL_TOKENS = True


def generate_response(model, tokenizer, system_input, context, user_input):
  # save parameters, input, and output
  # Format the input
  if model.name in bling_models:
    my_prompt = context + "\n" + user_input
    prompt = "\<human>\: " + my_prompt + "\n" + "\<bot>\:"
  elif model.name in intel_model:
    prompt = f"### System:\n{system_input}\n### User:\n{context}\n{user_input}\n### Assistant:\n"
  elif model.name in mistral_models:
    messages = [{'role': 'user', 'content': f'System: {system_input}\nContext: {context}\nUser query:{user_input}'}]
  else:
    prompt = f"System:\n{system_input}\nContext:\n{context}\nUser:\n{user_input}\nAssistant:\n"

  # Tokenize and encode the prompt
  if model.name in mistral_models:
    encoded_prompt = tokenizer.apply_chat_template(messages, return_tensors="pt").to('cuda')
  else:
    encoded_prompt = tokenizer.encode(prompt, return_tensors="pt").to('cuda') # add_special_tokens=False

  # Generate a response
  outputs = model.generate(encoded_prompt, max_new_tokens=MAX_NEW_TOKENS, do_sample=DO_SAMPLE,
                            num_beams=NUM_BEAMS, pad_token_id=tokenizer.eos_token_id,
                            eos_token_id=tokenizer.eos_token_id, temperature=TEMP)
  response = tokenizer.decode(outputs[0], skip_special_tokens=SKIP_SPECIAL_TOKENS)

  # Extract only the assistant's response
  if model.name in mistral_models:
    response = response.split('[/INST]')[-1]
    if response.endswith('</s>'):
      response = response[:-4]
  else:
    response = response.split("Assistant:\n")[-1]
  return response, params_log

In [9]:
system_input = """You are an assistant to potential students who are looking for information
                  about courses offered by the University of Gothenburg. You will be provided
                  relevant context and a question or instruction that you will respond to in a
                  professional manner. Base your answer on the given context."""

context = """The following text is from the syllabus for course DIT247.\n
The course gives an introduction to machine learning models and architectures used in
modern natural language processing (NLP) systems.
Rapid developments in machine learning have revolutionized the field of NLP,
including for commerically important applications such as translation, summarization,
and information extraction. However, natural language data exhibit a number of
peculiarities that make them more challenging to work with than many other types of
data commonly encountered in machine learning: natural language is discrete,
structured, and highly ambiguous. It is extremely diverse: not only are there thousands
of languages in the world, but in each language there is substantial variation in style and
genre. Furthermore, many of the phenomena encountered in language follow long-tail
statistical distributions, which makes the production of training data more costly. For
these reasons, machine learning architectures for NLP applications tend to be quite
different from those used in other fields.
The course covers the following broad areas:
Working practically with text data, including fundamental tasks such as
tokenization and word counting;
probabilistic models for text, such as topic models;
overview of the most common types of NLP applications;
architectures for representation in NLP models, including word embeddings,
convolutional and recurrent neural network, and attention models;
machine learning models for common types of NLP problems, mainly
categorization, sequence labeling, structured prediction and generation;
approaches to transfer learning in NLP."""

user_query = "Summarize the contents of the course DIT247."
user_query = "Does course DIT247 cover the transformer architecture?"
#user_query = "What should I know if I want to enroll in DIT247?"

# (OLD) Output from a somewhat successful run:
# 'Automation of natural language processing (NLP) systems.The course gives an introduction to machine learning models and architectures used in \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0modern natural languageprocessing (np) systems, as well as a brief overview of the most common types of NLP applications, and an overview of how they can be used to improve the performance of machine learning in NLP. \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0Context:'

In [ ]:
context = df[df['course_code'] == 'DIT862']['Course content'].iloc[0]
user_query = "Summarize the contents of course DIT862."
context

'passage: course code: dit862, course content the course gives an introduction to the theory of probability and statistics, data analysis using descriptive statistics and data visualization, and applications of probabilistic modeling in data science. in the course, the following broad areas will be covered: data analysis including descriptive statistics and data visualization probability theory including basic probability calculations, random variables, distributions statistical methods including point and interval estimates, hypothesis testing, regression probabilistic models in data science applications, for instance, naive bayes classifiers and topic models for text or hidden markov models for sequences sub-courses 1. take-home examination (hemtentamen), 4 credits grading scale: pass with distinction (vg), pass (g) and fail (u) 2. assignments (inlämningsuppgifter), 3.5 credits 42/ dit862 statistical methods for data science, 7.5 credits / statistiska metoder för data science, 7,5 hö

In [ ]:
context = df[df['course_code'] == 'DIT247']['Course content'].iloc[0]
user_query = 'What are the requirements for DIT247?'
#user_query = "What are the contents of course DIT247?"
context

'passage: course code: dit247, course content the course gives an introduction to machine learning models and architectures used in modern natural language processing (nlp) systems. rapid developments in machine learning have revolutionized the field of nlp, including for commerically important applications such as translation, summarization, and information extraction. however, natural language data exhibit a number of peculiarities that make them more challenging to work with than many other types of 2/ 4 dit247 machine learning for natural language processing, 7.5 credits / maskininlärning för språkteknologi, 7,5 högskolepoäng second cycle data commonly encountered in machine learning: natural language is discrete, structured, and highly ambiguous. it is extremely diverse: not only are there thousands of languages in the world, but in each language there is substantial variation in style and genre. furthermore, many of the phenomena encountered in language follow long-tail statistic

In [ ]:
df[df['course_codes'] == 'MMG710']['Entry requirements'].iloc[0]

'passage: course code: mmg710, entry requirements the prerequisites for the course fourier analysis are general entry requirements and the equivalent of 60 higher education credits in mathematics.'

In [ ]:
context = df[df['course_codes'] == 'MMG710']['Entry requirements'].iloc[0]
user_query = "What are the requirements for course mmg710?"

In [14]:
# test
response = generate_response(model, tokenizer, system_input, context, user_query)
response

' Yes, according to the provided syllabus, course DIT247 covers architectures for representation in NLP models, including attention models, which is a type of architecture used in transformer models. Therefore, the course should provide an introduction to transformer architectures for natural language processing.'

In [ ]:
response

' The requirements for course MMG710, Fourier Analysis, include meeting the general entry requirements of the University of Gothenburg and having the equivalent of 60 higher education credits in mathematics as prerequisites.'

In [15]:
model.name

'mistralai/Mistral-7B-Instruct-v0.2'